In [1]:
import urllib.request
from bs4 import BeautifulSoup as bs
import re
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
class Page:
    
    def __init__(self, myStart):
        self.start = myStart
        self.nodes = []
        self.edges = []
        self.centerTags = []

    # def findTags(node):


    def findAll(self, n):
        '''
        finds all nodes n branches away from center
        @param center is a string representing a wikipedia article title
        @param n is an int
        '''
        tier = 0
        findNext(self.start, 0, n)

    def findNext(center, tier, n):
        if(tier + 1 < n):
            fooNodes = findTier(center)
            for s in fooNodes:
                if( s not in nodes):
                    nodes.append(s)
                    edges.append( sorted([center, s]))
                    findNext(s, tier + 1, n)

    def findTier(center):
        '''
        finds all nodes 1 branch away from center
        @param center is a string representing a wikipedia article title
        '''
        nodeList = []
        response = urllib.request.urlopen('https://en.wikipedia.org/wiki/' + center)
        html = response.read()
        soup = bs(html,'lxml')
        for par in soup.find_all('p'):
            for link in par.find_all('a', href=True):
                url = link.get('href')
                if(url.startswith('/wiki/')) and (':' not in url):
                    if('#' in url):
                        hashLoc = url.find('#')
                        curSubj = url[6:hashLoc]
                    else:
                        curSubj = url[6:]
                    nodeList.append(curSubj)
        return nodeList

    def getNodes(self):
        return nodes
    def getEdges(self):
        return edges

In [6]:
pg = Page('science')
pg.findAll(3)

print('nodes------------------------------------------------------------------------------------------')
print(len(pg.getNodes()))
for e in pg.getNodes():
    print(e)
    
print('edges------------------------------------------------------------------------------------------')
print(len(pg.getEdges()))
for e in pg.getEdges():
    print(e)
    

NameError: name 'findNext' is not defined